
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



-------------------------------------------------------------------------------------------------------

# Pricing and Driver Earnings for a Two-Sided Mobility Platform: A Case of Amsterdam, the Netherlands

or 

# The Effects of Profit-Based Pricing on Driver Earnings and Performance of Two-Sided Mobility Platforms

# Abstract  

In this paper, we investigate how the  pricing of ride-pooling affects driver earnings. We also examine how profit-based setting affects these performance indicators. To this end, we applied a matching algorithm  to the case of ride-pooling and give a choice set to the driver for the case of Amsterdam, the Netherlands. For our simulation, we utilize an agent-based simulator reproducing the transport systems for two-sided mobility platforms (like Uber and Lyft) and applied three state-of-the-art pricing strategies such as <strong>profit maximization</strong>,  <strong>solo ride-hailing</strong>, and <strong>nearest pickup ride-pooling</strong>. We find that the profit maximization pricing strategy outperforms the other and traveler utility can be further improved by $\%X$ while reducing the total cost to serve the pooled rides. While offering a discount for profit maximization travel time is significantly higher $\%X$  than for private rides. 

-------------------------------------------------------------------------------------------------------

## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo ride-hailing</strong>

<strong>3. Nearest pickup ride-pooling</strong>

## Load ExMAS and MaaSSim


In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

## Delft, Netherlands

In [2]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 300 # reuqests (and passengers)
params.nV = 30 # vehicles



## Parameters for ExMAS

In [3]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [4]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

23-05-23 11:47:36-INFO-Initializing pairwise trip shareability between 300 and 300 trips.
23-05-23 11:47:36-INFO-creating combinations
23-05-23 11:47:36-INFO-89700	 nR*(nR-1)
23-05-23 11:47:37-INFO-Reduction of feasible pairs by 99.84%
23-05-23 11:47:37-INFO-Degree 2 	Completed
23-05-23 11:47:37-INFO-trips to extend at degree 2 : 402
23-05-23 11:47:37-INFO-At degree 2 feasible extensions found out of 29 searched
23-05-23 11:47:37-INFO-Degree 3 	Completed
23-05-23 11:47:37-INFO-trips to extend at degree 3 : 29
23-05-23 11:47:37-INFO-At degree 3 feasible extensions found out of 0 searched
23-05-23 11:47:37-INFO-Degree 4 	Completed
23-05-23 11:47:37-INFO-Max degree reached 4
23-05-23 11:47:37-INFO-Trips still possible to extend at degree 4 : 0


In [5]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

23-05-23 11:47:37-INFO-Matching 300 trips to 731 rides in order to min u_veh
23-05-23 11:47:38-INFO-Problem solution: Optimal. 
Total costs for single trips:         81,703 
reduced by matching to:               70,572


In [6]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],7.259500,392,1,[7.2595],"[0, 392]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44833192, 44764023]","[None, 0, 0]",node time req_id od 0 NaN...,392,3.92,5.88750,1.17750,4.7100
1,[1],3.627500,196,1,[3.6275],"[29, 196]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1552650889, 1584013932]","[None, 1, 1]",node time req_id od 0 ...,196,1.96,2.94150,0.58830,2.3532
2,[2],6.271500,339,1,[6.2715],"[302, 339]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44848435, 1563023404]","[None, 2, 2]",node time req_id od 0 ...,339,3.39,5.08500,1.01700,4.0680
3,[3],2.064000,111,1,[2.064],"[424, 111]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1258470957, 620834410]","[None, 3, 3]",node time req_id od 0 ...,111,1.11,1.67550,0.33510,1.3404
4,[4],6.521000,352,1,[6.521],"[431, 352]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 27082148, 4300758655]","[None, 4, 4]",node time req_id od 0 ...,352,3.52,5.28900,1.05780,4.2312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,"[117, 111, 116]",18.908650,802,32,"[10.532074999999999, 4.93175, 3.4448250000000002]","[5674.0, 81, 243, 78, 162, 238]","[117, 111, 116]","[111, 116, 117]",3,726,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3669180601, 44865433, 44845271, 4482965...","[None, 117, 111, 116, 111, 116, 117]",node time req_id od 0 ...,802,8.02,11.97900,2.39580,9.5832
727,"[117, 114, 124]",20.536625,866,32,"[10.72545, 5.1675, 4.643675]","[5733.5, 67, 296, 20, 272, 211]","[117, 114, 124]","[114, 124, 117]",3,727,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3669180601, 1838103225, 44826460, 44832...","[None, 117, 114, 124, 114, 124, 117]",node time req_id od 0 ...,866,8.66,12.67650,2.53530,10.1412
728,"[117, 114, 111]",20.776925,793,32,"[10.65265, 5.07195, 5.052325]","[5733.5, 67, 44, 286, 30, 366]","[117, 114, 111]","[114, 111, 117]",3,728,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3669180601, 1838103225, 44865433, 44832...","[None, 117, 114, 111, 114, 111, 117]",node time req_id od 0 ...,793,7.93,12.95775,2.59155,10.3662
729,"[117, 116, 124]",17.644600,787,32,"[10.181725, 3.458475, 4.0044]","[5675.0, 230, 92, 150, 104, 211]","[117, 116, 124]","[116, 124, 117]",3,729,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3669180601, 44845271, 44826460, 1421472...","[None, 117, 116, 124, 116, 124, 117]",node time req_id od 0 ...,787,7.87,11.69775,2.33955,9.3582


In [7]:
params.kpi = 1

In [8]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

23-05-23 11:47:43-INFO-Matching 300 trips to 731 rides in order to min u_veh
23-05-23 11:47:43-INFO-Problem solution: Optimal. 
Total costs for single trips:         81,703 
reduced by matching to:               70,572
0.5
0.5
0.5
23-05-23 11:47:48-CRITICAL-this is request 2 with [2, 547] available rides.
23-05-23 11:47:48-CRITICAL-ride 2 available [1]
23-05-23 11:47:48-CRITICAL-ride 547 available [1, 0]
23-05-23 11:47:48-CRITICAL-this is reuqest 2 with [2, 547] still available rides.
Profit Maximization
23-05-23 11:47:48-CRITICAL-vehicle 30 has 2 choices
0.5
0.5
23-05-23 11:47:48-CRITICAL-this is request 4 with [4, 421, 518, 542, 694] available rides.
23-05-23 11:47:48-CRITICAL-ride 4 available [1]
23-05-23 11:47:48-CRITICAL-ride 421 available [1, 0]
23-05-23 11:47:48-CRITICAL-ride 518 available [1, 0]
23-05-23 11:47:48-CRITICAL-ride 542 available [1, 0]
23-05-23 11:47:48-CRITICAL-ride 694 available [1, 0]
23-05-23 11:47:48-CRITICAL-this is reuqest 4 with [4, 421, 518, 542, 694] still

In [9]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM,REVENUE
veh,,,,,,,,,,,,,,,,,,,,,
1,12.0,0.0,False,0,0,7801.0,135.0,0,586.0,0,...,0,0,9148.0,2419.0,0,1150.0,1081.0,4498.0,32.007,2.9592
2,8.0,0.0,False,0,0,8517.0,60.0,0,295.0,0,...,0,0,10071.0,1520.0,0,896.0,595.0,7060.0,22.405,5.2272
3,13.0,0.0,False,0,0,10429.0,150.0,0,348.0,0,...,0,0,7043.0,1841.0,0,1398.0,888.0,2916.0,24.244,0.7992
4,11.0,0.0,False,0,0,9315.0,120.0,0,460.0,0,...,0,0,8061.0,2455.0,0,869.0,899.0,3838.0,31.489,4.5720
5,10.0,0.0,False,0,0,11325.0,90.0,0,227.0,0,...,0,0,7185.0,1484.0,0,988.0,617.0,4096.0,23.616,3.7236
6,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0,0,21599.0,0.0,0,0.0,0.0,21599.0,0.000,0.0000
7,15.0,0.0,False,0,0,8821.0,180.0,0,392.0,0,...,0,0,7223.0,3139.0,0,1394.0,1022.0,1668.0,39.494,3.7128
8,8.0,0.0,False,0,0,8051.0,75.0,0,271.0,0,...,0,0,11158.0,1370.0,0,434.0,586.0,8768.0,21.360,2.4804
9,0.0,0.0,False,0,0,867.0,0.0,0,0.0,0,...,0,0,20732.0,0.0,0,0.0,0.0,20732.0,0.000,0.0000


In [10]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,392.0,20.0,30.0,331.0,0.0,0.0,10.0,0,0,0,0,0,0,392.0,331.0,55.0,False
1,15.0,196.0,20.0,30.0,9980.0,0.0,29.0,10.0,0,0,0,0,0,0,196.0,9980.0,55.0,False
2,15.0,339.0,20.0,30.0,14.0,0.0,302.0,10.0,0,0,0,0,0,0,339.0,14.0,55.0,False
3,15.0,111.0,20.0,30.0,61.0,0.0,424.0,10.0,0,0,0,0,0,0,111.0,61.0,55.0,False
4,15.0,559.0,20.0,30.0,131.0,7.0,431.0,10.0,0,0,0,0,0,0,559.0,138.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,13.0,244.0,22.0,30.0,202.0,14235.0,2.0,10.0,0,0,0,0,0,0,244.0,14437.0,53.0,False
296,15.0,168.0,35.0,30.0,121.0,14128.0,126.0,10.0,0,0,0,0,0,0,168.0,14249.0,55.0,False
297,15.0,66.0,20.0,30.0,112.0,0.0,14313.0,10.0,0,0,0,0,0,0,66.0,112.0,55.0,False


In [11]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

KeyError: True

In [ ]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/pax{}.csv')